In [1]:
import nba_py
import datetime
import time
import nba_py.game
import nba_py.player
import nba_py.team
import pandas as pd
import numpy as np
import functools
import time
import pymysql
from sqlalchemy import create_engine
%matplotlib inline

In [21]:
conn = create_engine('mysql+pymysql://root:@localhost:3306/nba_stats')
player_list_18 = nba_py.player.PlayerList(season = '2017-18').info()['PERSON_ID']
print('Player list load complete!')
game_logs = pd.read_sql_table('game_logs', conn)
length_former = len(game_logs)

for p in player_list_18:
    for y in ['2014-15', '2015-16', '2016-17']:
        print(p, y, str(player_list_18[player_list_18 == p].index[0] + 1) + '/' + str(len(player_list_18)))
        game_logs = game_logs.append(nba_py.player.PlayerGameLogs(p, season = y).info())

'''
for p in player_list_18:
    game_logs = game_logs.append(nba_py.player.PlayerGameLogs(p, season = '2017-18').info())
'''
        
game_logs = game_logs.drop_duplicates(['Player_ID', 'Game_ID'])
length_new = len(game_logs)
print(str(length_new - length_former) + ' game logs append!')

game_logs.to_sql('game_logs', conn, index = False, if_exists = 'replace')
print(str(length_new) + ' game logs commit complete!')

Player list load complete!
203518 2014-15 1/543
203518 2015-16 1/543
203518 2016-17 1/543
203112 2014-15 2/543
203112 2015-16 2/543
203112 2016-17 2/543
203500 2014-15 3/543
203500 2015-16 3/543
203500 2016-17 3/543
1628389 2014-15 4/543
1628389 2015-16 4/543
1628389 2016-17 4/543
201167 2014-15 5/543
201167 2015-16 5/543
201167 2016-17 5/543
201582 2014-15 6/543
201582 2015-16 6/543
201582 2016-17 6/543
202332 2014-15 7/543
202332 2015-16 7/543
202332 2016-17 7/543
200746 2014-15 8/543
200746 2015-16 8/543
200746 2016-17 8/543
1628386 2014-15 9/543
1628386 2015-16 9/543
1628386 2016-17 9/543
1628443 2014-15 10/543
1628443 2015-16 10/543
1628443 2016-17 10/543
202730 2014-15 11/543
202730 2015-16 11/543
202730 2016-17 11/543
2754 2014-15 12/543
2754 2015-16 12/543
2754 2016-17 12/543
202329 2014-15 13/543
202329 2015-16 13/543
202329 2016-17 13/543
101187 2014-15 14/543
101187 2015-16 14/543
101187 2016-17 14/543
1626147 2014-15 15/543
1626147 2015-16 15/543
1626147 2016-17 15/543
2039

203076 2014-15 123/543
203076 2015-16 123/543
203076 2016-17 123/543
1627738 2014-15 124/543
1627738 2015-16 124/543
1627738 2016-17 124/543
202334 2014-15 125/543
202334 2015-16 125/543
202334 2016-17 125/543
201942 2014-15 126/543
201942 2015-16 126/543
201942 2016-17 126/543
203473 2014-15 127/543
203473 2015-16 127/543
203473 2016-17 127/543
1626155 2014-15 128/543
1626155 2015-16 128/543
1626155 2016-17 128/543
1627098 2014-15 129/543
1627098 2015-16 129/543
1627098 2016-17 129/543
203521 2014-15 130/543
203521 2015-16 130/543
203521 2016-17 130/543
2736 2014-15 131/543
2736 2015-16 131/543
2736 2016-17 131/543
1627767 2014-15 132/543
1627767 2015-16 132/543
1627767 2016-17 132/543
203476 2014-15 133/543
203476 2015-16 133/543
203476 2016-17 133/543
203915 2014-15 134/543
203915 2015-16 134/543
203915 2016-17 134/543
1628416 2014-15 135/543
1628416 2015-16 135/543
1628416 2016-17 135/543
1628422 2014-15 136/543
1628422 2015-16 136/543
1628422 2016-17 136/543
1628495 2014-15 137/54

2744 2014-15 241/543
2744 2015-16 241/543
2744 2016-17 241/543
2210 2014-15 242/543
2210 2015-16 242/543
2210 2016-17 242/543
201943 2014-15 243/543
201943 2015-16 243/543
201943 2016-17 243/543
201973 2014-15 244/543
201973 2015-16 244/543
201973 2016-17 244/543
101161 2014-15 245/543
101161 2015-16 245/543
101161 2016-17 245/543
1627744 2014-15 246/543
1627744 2015-16 246/543
1627744 2016-17 246/543
1626177 2014-15 247/543
1626177 2015-16 247/543
1626177 2016-17 247/543
201949 2014-15 248/543
201949 2015-16 248/543
201949 2016-17 248/543
2207 2014-15 249/543
2207 2015-16 249/543
2207 2016-17 249/543
1626169 2014-15 250/543
1626169 2015-16 250/543
1626169 2016-17 250/543
204020 2014-15 251/543
204020 2015-16 251/543
204020 2016-17 251/543
202325 2014-15 252/543
202325 2015-16 252/543
202325 2016-17 252/543
203999 2014-15 253/543
203999 2015-16 253/543
203999 2016-17 253/543
2563 2014-15 254/543
2563 2015-16 254/543
2563 2016-17 254/543
1627745 2014-15 255/543
1627745 2015-16 255/543
1

1627749 2015-16 358/543
1627749 2016-17 358/543
1627750 2014-15 359/543
1627750 2015-16 359/543
1627750 2016-17 359/543
203488 2014-15 360/543
203488 2015-16 360/543
203488 2016-17 360/543
1627846 2014-15 361/543
1627846 2015-16 361/543
1627846 2016-17 361/543
1626204 2014-15 362/543
1626204 2015-16 362/543
1626204 2016-17 362/543
203894 2014-15 363/543
203894 2015-16 363/543
203894 2016-17 363/543
2749 2014-15 364/543
2749 2015-16 364/543
2749 2016-17 364/543
203526 2014-15 365/543
203526 2015-16 365/543
203526 2016-17 365/543
203094 2014-15 366/543
203094 2015-16 366/543
203094 2016-17 366/543
201149 2014-15 367/543
201149 2015-16 367/543
201149 2016-17 367/543
203457 2014-15 368/543
203457 2015-16 368/543
203457 2016-17 368/543
203512 2014-15 369/543
203512 2015-16 369/543
203512 2016-17 369/543
1717 2014-15 370/543
1717 2015-16 370/543
1717 2016-17 370/543
1628373 2014-15 371/543
1628373 2015-16 371/543
1628373 2016-17 371/543
203994 2014-15 372/543
203994 2015-16 372/543
203994 20

202738 2016-17 475/543
202498 2014-15 476/543
202498 2015-16 476/543
202498 2016-17 476/543
202691 2014-15 477/543
202691 2015-16 477/543
202691 2016-17 477/543
202684 2014-15 478/543
202684 2015-16 478/543
202684 2016-17 478/543
1628414 2014-15 479/543
1628414 2015-16 479/543
1628414 2016-17 479/543
201229 2014-15 480/543
201229 2015-16 480/543
201229 2016-17 480/543
1626157 2014-15 481/543
1626157 2015-16 481/543
1626157 2016-17 481/543
200782 2014-15 482/543
200782 2015-16 482/543
200782 2016-17 482/543
202323 2014-15 483/543
202323 2015-16 483/543
202323 2016-17 483/543
1626167 2014-15 484/543
1626167 2015-16 484/543
1626167 2016-17 484/543
202327 2014-15 485/543
202327 2015-16 485/543
202327 2016-17 485/543
2757 2014-15 486/543
2757 2015-16 486/543
2757 2016-17 486/543
1627755 2014-15 487/543
1627755 2015-16 487/543
1627755 2016-17 487/543
202685 2014-15 488/543
202685 2015-16 488/543
202685 2016-17 488/543
1627756 2014-15 489/543
1627756 2015-16 489/543
1627756 2016-17 489/543
16